In [ ]:
# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git

In [2]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px

In [3]:
person = pd.read_stata(
    "https://www2.census.gov/programs-surveys/supplemental-poverty-measure/datasets/spm/spm_2019_pu.dta",
    columns=[
        "serialno",
        "sporder",
        "wt",
        "age",
        "race",
        "hispanic",
        "spm_id",
        "spm_povthreshold",
        "spm_resources",
        "st",
        "puma"
    ],
)

In [4]:
# Cleanup
person.columns = person.columns.str.lower()
person = person.rename(columns={'serialno': 'serial', 'sporder':'pernum'})

In [5]:
person = person.astype({"serial":'int', "pernum":'int',
                          "wt":'int', "age":'int',
                          "spm_id":'int', "spm_povthreshold":'int',
                          "spm_resources": "int"}) 

In [6]:
# Sort to just Maryland
person = person[person['st'] == 24]

In [7]:
# Replace NIUs
person = person.replace(9999999,0)

In [8]:
# Define age groups
person['child'] = person.age < 18
person['young_child'] = person.age < 5
person['baby'] = person.age == 0

In [9]:
# Use groupby to calculate total babies, young children, and children in each spm unit
spmu = person.groupby(['spm_id'])[['child', 'young_child', 'baby']].sum()
spmu.columns = ['spm_children', 'spm_young_children', 'spm_babies']
# merge back onto the person dataframe
person = person.merge(spmu, left_on =['spm_id'], right_index=True)

In [10]:
# 

def pov(reform, age_group):
  if age_group == 'All':
      tp = person.copy(deep=True)

  if age_group == 'Young Child':
    tp = person[person['age'] < 5]

  if age_group == 'Child':
    tp = person[person['age'] < 18]
  
  if age_group == 'Adult':
    tp = person[person['age'] > 17]

  if reform == 'All Children 100':
    tp['total_ca'] = tp.spm_children * 100 * 12

  if reform == 'All Children 200':
    tp['total_ca'] = tp.spm_children * 200 * 12
  
  if reform == 'Young Children 100':
    tp['total_ca'] = tp.spm_young_children * 100 * 12
  
  if reform == 'Young Children 200':
    tp['total_ca'] = tp.spm_young_children * 200 * 12

  tp['new_resources'] = tp.total_ca + tp.spm_resources	
  tp['still_poor'] = tp.new_resources < tp.spm_povthreshold

  # populations
  population = (tp.wt).sum()

  # orginal poverty rates
  tp['poor'] = tp.spm_resources < tp.spm_povthreshold

  total_poor = (tp.poor * tp.wt).sum()
  total_pov_rate = (total_poor / population)

  # new poverty rates
  new_total_poor = (tp.still_poor * tp.wt).sum()
  new_total_pov_rate = (new_total_poor / population)

  # percent change
  total_pov_change = ((new_total_poor - total_poor) / (total_poor) * 100).round(1)
  
  return pd.Series([total_pov_change,
          population,
          total_pov_rate,
          new_total_pov_rate,
                    ])

In [11]:
summary = mdf.cartesian_product({
                       'reform':['All Children 100', 'All Children 200', 'Young Children 100', 'Young Children 200'],
                       'age_group': ['All', 'Young Child', 'Child', 'Adult']})

In [12]:
def pov_row(row):
  return pov(row.reform, row.age_group)

In [13]:
summary[['total_pov_change',
         'population',
          'total_pov_rate',
          'new_total_pov_rate',]] = summary.apply(pov_row, axis=1)

In [14]:
summary.to_csv('maryland_age.csv')

In [15]:
all_100 = summary[summary['reform'] == 'All Children 100']
all_200 = summary[summary['reform'] == 'All Children 200']
young_100 = summary[summary['reform'] == 'Young Children 100']
young_200 = summary[summary['reform'] == 'Young Children 200']

In [16]:
fig = px.bar(all_100, x="age_group", y="total_pov_change",
              labels={
                     "total_pov_change": "Change in SPM poverty rate",
                     "age_group": "Age Group",
                     "child_allowance": "Child Allowance"
                 },
                title="Change in child poverty by age group", 
             color='age_group', 
             text='total_pov_change',
             color_discrete_map={
                                 'All': '#003f5c',
                                 'Adult': '#5886a5',
                                 'Child': '#5886a5',
                                 'Young Child': '#5886a5'}, height=700, width=700)

fig.update_layout(showlegend=False, yaxis_ticksuffix='%')
fig.update_traces(texttemplate='%{text}%')
fig.show()

In [17]:
fig = px.bar(all_200, x="age_group", y="total_pov_change",
              labels={
                     "total_pov_change": "Change in SPM poverty rate",
                     "age_group": "Age Group",
                     "child_allowance": "Child Allowance"
                 },
                title="Change in child poverty by age group", 
             color='age_group', 
             text='total_pov_change',
             color_discrete_map={
                                 'All': '#003f5c',
                                 'Adult': '#5886a5',
                                 'Child': '#5886a5',
                                 'Young Child': '#5886a5'}, height=700, width=700)

fig.update_layout(showlegend=False, yaxis_ticksuffix='%')
fig.update_traces(texttemplate='%{text}%')
fig.show()

In [18]:
fig = px.bar(young_100, x="age_group", y="total_pov_change",
              labels={
                     "total_pov_change": "Change in SPM poverty rate",
                     "age_group": "Age Group",
                     "child_allowance": "Child Allowance"
                 },
                title="Change in child poverty by age group", 
             color='age_group', 
             text='total_pov_change',
             color_discrete_map={
                                 'All': '#003f5c',
                                 'Adult': '#5886a5',
                                 'Child': '#5886a5',
                                 'Young Child': '#5886a5'}, height=700, width=700)

fig.update_layout(showlegend=False, yaxis_ticksuffix='%')
fig.update_traces(texttemplate='%{text}%')
fig.show()

In [19]:
fig = px.bar(young_200, x="age_group", y="total_pov_change",
              labels={
                     "total_pov_change": "Change in SPM poverty rate",
                     "age_group": "Age Group",
                     "child_allowance": "Child Allowance"
                 },
                title="Change in child poverty by age group", 
             color='age_group', 
             text='total_pov_change',
             color_discrete_map={
                                 'All': '#003f5c',
                                 'Adult': '#5886a5',
                                 'Child': '#5886a5',
                                 'Young Child': '#5886a5'}, height=700, width=700)

fig.update_layout(showlegend=False, yaxis_ticksuffix='%')
fig.update_traces(texttemplate='%{text}%')
fig.show()